In [1]:
import cv2
import os
import time

def extract_frames(video_path, output_dir="frames"):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    fps = int(cap.get(cv2.CAP_PROP_FPS) or 30)
    print(f"🎞️ Extracting frames from '{video_path}'...")
    while True:
        ret, frame = cap.read()
        if not ret: break
        cv2.imwrite(f"{output_dir}/frame_{count:06d}.jpg", frame)
        count += 1
    cap.release()
    print(f"✅ Extracted {count} frames at {fps} FPS")
    return count, fps

if __name__ == "__main__":
    start = time.time()
    total, fps = extract_frames("jumbled_video.mp4")
    print(f"⏱️ Done in {time.time()-start:.2f}s")

🎞️ Extracting frames from 'jumbled_video.mp4'...
✅ Extracted 0 frames at 30 FPS
⏱️ Done in 0.09s


In [4]:
# =====================================
# DAY 2: GPU Detection + SSIM Kernel
# =====================================
import torch
import torch.nn.functional as F

# --------------------------
# 🧠 1. Detect and Set Device
# --------------------------
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("🚀 Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("⚙️ Using CPU (no GPU found)")

# -----------------------------------------
# 🧩 2. Create Gaussian Kernel for SSIM
# -----------------------------------------
def gaussian_kernel(size=11, sigma=1.5, channels=3):
    """Creates a 2D Gaussian kernel expanded across channels."""
    coords = torch.arange(size, dtype=torch.float32) - size // 2
    grid = coords ** 2
    kernel_1d = torch.exp(-grid / (2 * sigma ** 2))
    kernel_1d /= kernel_1d.sum()
    kernel_2d = kernel_1d[:, None] @ kernel_1d[None, :]
    kernel_2d = kernel_2d.expand(channels, 1, size, size).to(device)
    return kernel_2d

# -----------------------------------------
# 🧮 3. SSIM Function (basic building block)
# -----------------------------------------
def ssim_torch(img1, img2, kernel, C1=0.01**2, C2=0.03**2):
    """Compute SSIM between two images using Gaussian kernel (Torch)."""
    mu1 = F.conv2d(img1, kernel, padding=kernel.shape[-1] // 2, groups=img1.shape[1])
    mu2 = F.conv2d(img2, kernel, padding=kernel.shape[-1] // 2, groups=img2.shape[1])

    mu1_sq = mu1 ** 2
    mu2_sq = mu2 ** 2
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, kernel, padding=kernel.shape[-1] // 2, groups=img1.shape[1]) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, kernel, padding=kernel.shape[-1] // 2, groups=img2.shape[1]) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, kernel, padding=kernel.shape[-1] // 2, groups=img1.shape[1]) - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / (
        (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
    )
    return ssim_map.mean()

# -----------------------------------------
# ✅ 4. Example Test (optional)
# -----------------------------------------
if __name__ == "__main__":
    # Create two dummy images (360x640 RGB)
    img1 = torch.rand(1, 3, 360, 640).to(device)
    img2 = torch.rand(1, 3, 360, 640).to(device)
    kernel = gaussian_kernel()
    val = ssim_torch(img1, img2, kernel)
    print(f"🔍 Test SSIM Value: {val.item():.4f}")


⚙️ Using CPU (no GPU found)
🔍 Test SSIM Value: 0.0162


In [1]:
# ===============================
# 🧩 Day 3: Frame Similarity Matrix (Histogram + Flow + SSIM)
# ===============================
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def combined_similarity(p, frames_resized, tensors_resized, device, use_ssim=True):
    i, j = p
    f1, f2 = frames_resized[i], frames_resized[j]

    # --- Color histogram similarity ---
    h1 = cv2.calcHist([f1], [0,1,2], None, [8,8,8], [0,256]*3)
    h2 = cv2.calcHist([f2], [0,1,2], None, [8,8,8], [0,256]*3)
    hist_score = (cv2.compareHist(h1, h2, cv2.HISTCMP_CORREL) + 1) / 2

    # --- Optical flow similarity ---
    gray1, gray2 = cv2.cvtColor(f1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(f2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    flow_score = np.clip(1 / (1 + np.mean(np.sqrt(flow[...,0]**2 + flow[...,1]**2))), 0, 1)

    # --- SSIM (GPU only) ---
    ssim_score = 0.0
    if use_ssim and tensors_resized[i] is not None and tensors_resized[j] is not None:
        with torch.no_grad():
            ssim_score = ssim_torch(tensors_resized[i], tensors_resized[j])
        ssim_score = np.clip(ssim_score, 0, 1)

    # --- Weighted combination ---
    w_ssim = 0.5 if use_ssim else 0.0
    w_hist = 0.45 if use_ssim else 0.7
    w_flow = 0.05 if use_ssim else 0.3

    return i, j, w_ssim * ssim_score + w_hist * hist_score + w_flow * flow_score


def build_similarity_matrix(frames_resized, tensors_resized, device):
    n = len(frames_resized)
    sim = np.zeros((n, n))
    pairs = [(i, j) for i in range(n) for j in range(i+1, n)]
    use_ssim = device.type != "cpu"

    print("🧮 Building frame similarity matrix...")

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(
            lambda args: combined_similarity(*args),
            [(p, frames_resized, tensors_resized, device, use_ssim) for p in pairs]
        ), total=len(pairs), ncols=100))

    for i, j, val in results:
        sim[i][j] = sim[j][i] = val

    print("✅ Similarity matrix built successfully.")
    return sim


In [4]:
#DAY4
import networkx as nx

def tsp_order(similarity):
    n = len(similarity)
    G = nx.Graph()
    for i in range(n):
        for j in range(i+1,n):
            G.add_edge(i,j,weight=1-similarity[i][j])
    order=[0]; visited={0}
    cur=0
    print("🔀 Reconstructing order...")
    for _ in range(n-1):
        best=-1; min_cost=float('inf')
        for neighbor in G.neighbors(cur):
            if neighbor not in visited and G[cur][neighbor]['weight']<min_cost:
                min_cost=G[cur][neighbor]['weight']; best=neighbor
        if best!=-1:
            order.append(best); visited.add(best); cur=best
    return order

def detect_direction(frames_resized, order):
    reverse = order[::-1]
    def motion_score(seq):
        total=0
        for i in range(len(seq)-1):
            f1=cv2.cvtColor(frames_resized[seq[i]],cv2.COLOR_BGR2GRAY)
            f2=cv2.cvtColor(frames_resized[seq[i+1]],cv2.COLOR_BGR2GRAY)
            flow=cv2.calcOpticalFlowFarneback(f1,f2,None,0.5,3,15,3,5,1.2,0)
            total+=np.mean(np.sqrt(flow[...,0]**2+flow[...,1]**2))
        return total
    return order if motion_score(order)>=motion_score(reverse) else reverse
